In [ ]:
# -*- coding: utf-8 -*-
"""Assistente de Dublagem de Vídeo"""

# Célula 1: Instalação de dependências e configuração inicial
#@title 🎬 Dublagem de Vídeo: Instalação e Introdução { display-mode: "form" }
#@markdown ## Bem-vindo ao Assistente de Dublagem de Vídeos!
#@markdown Este script automatiza a dublagem de vídeos do inglês para o português brasileiro usando tecnologias de ponta em IA.
#@markdown ### 🛠️ Tecnologias Utilizadas:
#@markdown - **FFmpeg**: Para manipulação de áudio e vídeo
#@markdown - **Whisper**: Transcrição de áudio da OpenAI
#@markdown - **Google Cloud Text-to-Speech**: Síntese de voz de alta qualidade
#@markdown - **Google Cloud Translate**: Tradução precisa de texto
#@markdown - **MoviePy & Pydub**: Processamento de áudio e vídeo em Python
#@markdown ### 📚 Repositório GitHub: (Exemplo - substitua pelo seu repositório se houver)
#@markdown [github.com/seu-usuario/seu-repositorio](https://github.com/seu-usuario/seu-repositorio)
#@markdown ### 🚀 Instalação de Dependências:
#@markdown Executar esta célula instalará todas as dependências necessárias.

!apt-get update && apt-get install -y ffmpeg
!pip install --quiet pydub moviepy google-cloud-texttospeech google-cloud-translate openai-whisper spacy spacy_syllables tqdm

!python -m spacy download en_core_web_sm
!python -m spacy download pt_core_news_sm

from google.colab import drive
drive.mount('/content/drive', force_remount=True)

print("Configuração inicial concluída. Você está pronto para começar a dublar seus vídeos!")

In [ ]:
# Célula 2: Instruções e preparação
#@title 🎬 Preparação para Dublagem { display-mode: "form" }
#@markdown ## Bem-vindo ao Assistente de Dublagem de Vídeos!
#@markdown Este script irá dublar seu vídeo do inglês para o português brasileiro.
#@markdown ### Passo 1: Configurar as Credenciais do Google Cloud
#@markdown 1. Acesse o [Console do Google Cloud](https://console.cloud.google.com/)
#@markdown 2. Crie um novo projeto ou selecione um existente
#@markdown 3. Ative as APIs: Cloud Text-to-Speech e Cloud Translation
#@markdown 4. Crie uma chave de conta de serviço e baixe o arquivo JSON
#@markdown 5. Renomeie o arquivo para `google_cloud_credentials.json`
#@markdown 6. Faça upload do arquivo para a pasta `Dubbing` no seu Google Drive
#@markdown ### Passo 2: Preparar o Vídeo
#@markdown 1. Faça upload do vídeo que deseja dublar para a pasta `Dubbing` no seu Google Drive
#@markdown 2. O script só dubla um arquivo por vez
#@markdown ### Passo 3: Escolher a Voz (Configurações abaixo)

print("Preparação concluída. Vamos começar!")

Preparação concluída. Vamos começar!


In [ ]:
# Célula 3: Importações e funções auxiliares
#@title 🔧 Configurações e Funções Auxiliares { display-mode: "form" }

import os
import uuid
import subprocess
from pydub import AudioSegment
from moviepy.editor import VideoFileClip, AudioFileClip
from google.cloud import texttospeech
from google.cloud import translate_v2 as translate
import whisper
import spacy
from spacy_syllables import SpacySyllables
from tqdm import tqdm
import tempfile
import re
from google.oauth2 import service_account
import json
import time

# Configurações da interface do Colab
voz_selecionada = "pt-BR-Neural2-B" #@param ["pt-BR-Neural2-A", "pt-BR-Neural2-B", "pt-BR-Neural2-C", "pt-BR-Standard-A", "pt-BR-Standard-B", "pt-BR-Standard-C", "pt-BR-Wavenet-A", "pt-BR-Wavenet-B", "pt-BR-Wavenet-C"]
#@markdown [Ouça amostras das vozes aqui](https://cloud.google.com/text-to-speech/docs/voices)
modelo_whisper = "tiny" #@param ["tiny", "base", "small", "medium", "large-v3"]


def listar_videos():
    video_path = '/content/drive/My Drive/Dubbing/'
    videos = [f for f in os.listdir(video_path) if f.endswith('.mp4')]
    for i, video in enumerate(videos):
        print(f"{i+1}. {video}")
    return videos

def carregar_video():
    videos = listar_videos()
    escolha = int(input("Escolha o número do vídeo: "))
    video_path = '/content/drive/My Drive/Dubbing/'
    return os.path.join(video_path, videos[escolha-1])

print("Funções auxiliares definidas.")

Funções auxiliares definidas.


In [ ]:
# Célula 4: Funções principais
#@title 🎭 Funções de Dublagem { display-mode: "form" }
def extract_audio_from_video(video_file):
    try:
        print("Extraindo faixa de áudio")
        audio_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3").name
        command = [
            'ffmpeg',
            '-i', video_file,
            '-vn',
            '-acodec', 'libmp3lame',
            '-b:a', '64k',
            '-y',
            audio_file
        ]
        subprocess.run(command, check=True, capture_output=True, text=True)
        print("Áudio extraído com sucesso.")
        return audio_file
    except subprocess.CalledProcessError as e:
        print(f"Erro ao extrair áudio do vídeo (código de retorno {e.returncode}): {e.stderr}")
        return None
    except Exception as e:
        print(f"Erro ao extrair áudio do vídeo: {e}")
        return None

def transcribe_audio(audio_file, source_language):
    try:
        print("Transcrevendo faixa de áudio")
        model = whisper.load_model(modelo_whisper, device="cpu")
        trans = model.transcribe(audio_file, language=source_language, fp16=False)
        return trans
    except Exception as e:
        print(f"Erro ao transcrever áudio: {e}")
        return None

def translate_text(texts, target_language, credentials):
    try:
        translate_client = translate.Client(credentials=credentials)
        results = []
        for i in tqdm(range(0, len(texts), 100), desc="Traduzindo textos"):
            batch = texts[i:i+100]
            batch_results = translate_client.translate(batch, target_language=target_language)
            results.extend(batch_results)
        return [result['translatedText'] for result in results]
    except Exception as e:
        print(f"Erro ao traduzir textos: {e}")
        return None

def create_audio_from_text(text, target_language, target_voice, credentials):
    audio_file = "translated_" + str(uuid.uuid4()) + ".mp3"
    try:
        client = texttospeech.TextToSpeechClient(credentials=credentials)
        input_text = texttospeech.SynthesisInput(text=text)
        voice = texttospeech.VoiceSelectionParams(
            language_code=target_language,
            name=target_voice
        )
        audio_config = texttospeech.AudioConfig(
            audio_encoding=texttospeech.AudioEncoding.MP3,
            speaking_rate=1.1,
            sample_rate_hertz=22050
        )
        response = client.synthesize_speech(
            input=input_text, voice=voice, audio_config=audio_config
        )
        with open(audio_file, "wb") as out:
            out.write(response.audio_content)
        return audio_file
    except Exception as e:
        if os.path.isfile(audio_file):
            os.remove(audio_file)
        print(f"Erro ao criar áudio a partir do texto: {e}")
        return None

def merge_audio_files(transcription, source_language, target_language, target_voice, credentials):
    temp_files = []
    try:
        nlp = spacy.load("en_core_web_sm")
        nlp.add_pipe("syllables", after="tagger")
        merged_audio = AudioSegment.silent(duration=0)
        sentences = [segment["text"] for segment in transcription["segments"]]

        print("Traduzindo frases")
        translated_texts = translate_text(sentences, target_language, credentials)
        if translated_texts is None:
            raise Exception("Falha na tradução")

        print("Criando faixa de áudio traduzida")
        for translated_text in tqdm(translated_texts, desc="Gerando áudio"):
            translated_text = re.sub(r'^OK\s*', '', translated_text, flags=re.IGNORECASE)
            translated_audio_file = create_audio_from_text(translated_text, target_language, target_voice, credentials)
            if translated_audio_file is None:
                raise Exception("Falha na criação do áudio")
            temp_files.append(translated_audio_file)
            translated_audio = AudioSegment.from_mp3(translated_audio_file)
            merged_audio += translated_audio

        return merged_audio
    except Exception as e:
        print(f"Erro ao mesclar arquivos de áudio: {e}")
        return None
    finally:
        for file in temp_files:
            try:
                os.remove(file)
            except Exception as e:
                print(f"Erro ao remover arquivo temporário {file}: {e}")

def replace_audio_in_video(video_file, new_audio, output_filename):
    try:
        print("Substituindo áudio no vídeo")
        temp_audio_file = tempfile.NamedTemporaryFile(delete=False, suffix=".mp3")
        new_audio.export(temp_audio_file.name, format="mp3", bitrate="64k")

        command = [
            'ffmpeg',
            '-i', video_file,
            '-i', temp_audio_file.name,
            '-c:v', 'copy',
            '-c:a', 'aac',
            '-b:a', '64k',
            '-map', '0:v:0',
            '-map', '1:a:0',
            '-y',
            output_filename
        ]
        subprocess.run(command, check=True, stdout=subprocess.DEVNULL, stderr=subprocess.DEVNULL)
        print(f"Vídeo dublado salvo como {output_filename}")
    except subprocess.CalledProcessError as e:
        print(f"Erro ao substituir áudio no vídeo (código de retorno {e.returncode}): {e.stderr}")
    except Exception as e:
        print(f"Erro ao substituir áudio no vídeo: {e}")
    finally:
        if os.path.isfile(temp_audio_file.name):
            os.remove(temp_audio_file.name)

print("Funções principais definidas.")

Funções principais definidas.


In [ ]:
# Célula 5: Função principal e execução
#@title 🚀 Iniciar Dublagem { display-mode: "form" }
def dub_video(video_file, target_voice, credentials):
    start_time = time.time()
    audio_file = extract_audio_from_video(video_file)

    if audio_file:
        try:
            transcription = transcribe_audio(audio_file, "en")
            if transcription:
                merged_audio = merge_audio_files(transcription, "en", "pt-BR", target_voice, credentials)
                if merged_audio:
                    base_name, ext = os.path.splitext(os.path.basename(video_file))
                    safe_base_name = re.sub(r'[\\/*?:"<>|]', "", base_name) # Remove caracteres inválidos
                    output_filename = os.path.join('/content/drive/My Drive/Dubbing/Results', f"{safe_base_name}__pt{ext}")
                    replace_audio_in_video(video_file, merged_audio, output_filename)
                    end_time = time.time()
                    print(f"Processamento do vídeo {video_file} concluído em {end_time - start_time:.2f} segundos!")
                else:
                    print(f"Falha ao dublar o vídeo {video_file}. merged_audio é None.")
        except Exception as e:
            print(f"Erro durante o processamento do vídeo {video_file}: {e}")
        finally:
            if os.path.exists(audio_file):
                os.remove(audio_file)

def main():
    print("Iniciando o processo de dublagem...")

    dubbing_folder = '/content/drive/My Drive/Dubbing/'
    credentials_path = os.path.join(dubbing_folder, 'google_cloud_credentials.json')
    if not os.path.exists(credentials_path):
        print("Erro: Arquivo de credenciais não encontrado.")
        print("Por favor, faça upload do arquivo 'google_cloud_credentials.json' para a pasta 'Dubbing' no seu Google Drive.")
        return

    try:
        credentials = service_account.Credentials.from_service_account_file(
            credentials_path,
            scopes=['https://www.googleapis.com/auth/cloud-platform']
        )
    except Exception as e:
        print(f"Erro ao carregar credenciais: {e}")
        return

    video_path = '/content/drive/My Drive/Dubbing/'
    os.makedirs(os.path.join(video_path, 'Results'), exist_ok=True)

    start_time = time.time()

    for filename in os.listdir(video_path):
        if filename.endswith(".mp4"):
            video_file = os.path.join(video_path, filename)
            try:
                dub_video(video_file, voz_selecionada, credentials)
            except Exception as e:
                print(f"Erro ao processar o vídeo {filename}: {e}")

    end_time = time.time()
    print(f"Processamento total concluído em {end_time - start_time:.2f} segundos!")

if __name__ == "__main__":
    main()